In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import *

train = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-apr-2021/test.csv')
y = train.Survived

print(np.sum(train.PassengerId == test.PassengerId))

train.drop(['PassengerId', 'Survived', 'Name', 'Ticket'], axis=1, inplace=True)
test.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)

train.head()

In [ ]:
target_column = 'Age'
train[train[target_column].notnull()].info()

In [ ]:
columns_to_fit = []
for key in train.keys():
    if key == target_column: continue
    if train[target_column].notnull().sum() == train[train[target_column].notnull()][key].notnull().sum():
        columns_to_fit.append(key)

columns_to_fit

In [ ]:
not_append, to_append = [], []
for column in columns_to_fit:
    if train[column].dtype == np.float64:
        not_append.append(column)
    else:
        to_append.append(column)
print(not_append)
print(to_append)

In [ ]:
from sklearn.preprocessing import *

onehotencoder = OneHotEncoder()
categorical = train[train[target_column].notnull()][to_append]
X = onehotencoder.fit_transform(categorical).toarray()

In [ ]:
X

In [ ]:
X = np.concatenate((X, train[train[target_column].notnull()][not_append]), axis=1)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

from sklearn.metrics import mean_squared_error, accuracy_score

model = DecisionTreeRegressor()
# model = LinearRegression()
model.fit(X, train[train[target_column].notnull()][target_column])
yp = model.predict(X)

print(mean_squared_error(yp, train[train[target_column].notnull()][target_column]))

In [ ]:
# Xt = onehotencoder.transform(train[train[target_column].isnull()][to_append]).toarray()
# Xt = np.concatenate((Xt, train[train[target_column].isnull()][not_append]), axis=1)

# ypt = model.predict(Xt)

Xt = onehotencoder.transform(train[to_append]).toarray()
Xt = np.concatenate((Xt, train[not_append]), axis=1)

ypt = model.predict(Xt)

In [ ]:
result = train[target_column].isnull() * ypt

In [ ]:
import matplotlib.pyplot as plt

imputer = SimpleImputer(missing_values=np.nan, strategy = 'mean')
imputer = imputer.fit(train[['Age']])
imputed_mean = imputer.transform(train[['Age']])

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 3))

axes[0].hist(imputed_mean, bins=50)
axes[1].hist(train["Age"], bins=50)
axes[2].hist(train[target_column] + result, bins=50)

fig.tight_layout()

#Age clearly has three age distributions, putting the average or median in place of 
#nan may not be the best strategy
#I'll separate into categories during training
#What to put in place of NAN? 
#forms of imput missing values
#We will use Regression on the other features

In [ ]:
train.fillna({target_column: 0}, inplace=True)
train[target_column] = train[target_column] + result

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import *

#Impute with regression from other features
def doMethodImpute(data, target_column, model=DecisionTreeRegressor):
    class doMethodImputeIntern():
        def __init__(self, target_column, not_append, to_append, onehotencoder, model):
            self.target_column = target_column
            self.not_append = not_append
            self.to_append = to_append
            self.onehotencoder = onehotencoder
            self.model = model

        def transform(self, new_data):
            Xt = self.onehotencoder.transform(new_data[self.to_append]).toarray()
            Xt = np.concatenate((Xt, new_data[self.not_append]), axis=1)
            ypt = self.model.predict(Xt)

            result = new_data[self.target_column].isnull() * ypt
            new_data.fillna({self.target_column: 0}, inplace=True)
            new_data[self.target_column] = new_data[self.target_column] + result
            return self

    columns_to_fit = []
    for key in data.keys():
        if key == target_column: continue
        if data[target_column].notnull().sum() == data[data[target_column].notnull()][key].notnull().sum():
            columns_to_fit.append(key)

    not_append, to_append = [], []
    for column in columns_to_fit:
        if data[column].dtype == np.float64:
            not_append.append(column)
        else:
            to_append.append(column)

    onehotencoder = OneHotEncoder()
    categorical = data[data[target_column].notnull()][to_append]
    X = onehotencoder.fit_transform(categorical).toarray()
    X = np.concatenate((X, data[data[target_column].notnull()][not_append]), axis=1)

    model = model()
    model.fit(X, data[data[target_column].notnull()][target_column])
    yp = model.predict(X)

    Xt = onehotencoder.transform(data[to_append]).toarray()
    Xt = np.concatenate((Xt, data[not_append]), axis=1)
    ypt = model.predict(Xt)

    result = data[target_column].isnull() * ypt
    data.fillna({target_column: 0}, inplace=True)
    data[target_column] = data[target_column] + result
    return doMethodImputeIntern(target_column, not_append, to_append, onehotencoder, model)

In [ ]:
imputer = doMethodImpute(train, 'Age')
imputer.transform(test)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
imputer = doMethodImpute(train, 'Fare')
imputer.transform(test)

In [ ]:
train.info()

In [ ]:
# train['Age'].interpolate(method='linear', inplace=False, limit_direction="both")

In [ ]:
train['Embarked'].hist()

In [ ]:
train.fillna({'Cabin': "NE"}, inplace=True)
test.fillna({'Cabin': "NE"}, inplace=True)

train.fillna({'Embarked': "NE"}, inplace=True)
test.fillna({'Embarked': "NE"}, inplace=True)

In [ ]:
for data in [train, test]:
    cabins_vec = []
    for i in data['Cabin']:
        if str(i) == 'NE':
            cabins_vec.append("NE")
        else:
            cabins_vec.append(i[0])
    data['Cabin'] = pd.Series(cabins_vec)

In [ ]:
for data in [train, test]:
    embk_vec = []
    for i in data['Embarked']:
        if str(i) == 'NE':
            embk_vec.append("NE")
        else:
            embk_vec.append(i)
    data['Embarked'] = pd.Series(embk_vec)

In [ ]:
train['Cabin'], train['Embarked']

In [ ]:
train['Cabin'].hist()

In [ ]:
train['Embarked'].hist()

In [ ]:
train.info()

In [ ]:
## Other options for Impute
# train.fillna({'Cabin': 0}, inplace=True)
# test.fillna({'Cabin': 0}, inplace=True)

# imputer = SimpleImputer(missing_values=np.nan, strategy = 'mean')
# imputer = imputer.fit(train[['Age']])
# train[['Age']] = imputer.transform(train[['Age']])
# test[['Age']] = imputer.transform(test[['Age']])

# #usando a mediana porque Fare varia muito, é skew
# #log também ajuda a reduzir skewness
# imputer = SimpleImputer(missing_values=np.nan, strategy = 'median')
# imputer = imputer.fit(train[['Fare']])
# train[['Fare']] = imputer.transform(train[['Fare']])
# test[['Fare']] = imputer.transform(test[['Fare']])

# train.fillna('notEval', inplace=True)
# test.fillna('notEval', inplace=True)

# train[["Fare"]] = train[["Fare"]].apply(pd.to_numeric)
# test[["Fare"]] = test[["Fare"]].apply(pd.to_numeric)

In [ ]:
train.Age.hist()

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

#DISCRETIZATION of Age and Fare
enc = KBinsDiscretizer(n_bins=10, encode='onehot')
enc = enc.fit(train['Age'].to_numpy().reshape(-1, 1))
Age_discretized_train = enc.transform(train['Age'].to_numpy().reshape(-1, 1))
Age_discretized_test = enc.transform(test['Age'].to_numpy().reshape(-1, 1))
# print(Age_discretized_train)

#Fare we are going to separate at intervals because there is high concentration in a region,
#and log because Fare has an asymmetric distribution 
before_normalization_train_fare = train['Fare'].copy()
train['Fare'] = np.log(train['Fare'])
test['Fare'] = np.log(test['Fare'])
enc = KBinsDiscretizer(n_bins=20, encode='onehot')
enc = enc.fit(train['Fare'].to_numpy().reshape(-1, 1))
Fare_discretized_train = enc.transform(train['Fare'].to_numpy().reshape(-1, 1))
Fare_discretized_test = enc.transform(test['Fare'].to_numpy().reshape(-1, 1))


In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))

axes[0].hist(before_normalization_train_fare, bins=50)
axes[1].hist(train["Fare"], bins=50)
fig.tight_layout()

plt.show()


In [ ]:
from sklearn.preprocessing import *
categorical_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Cabin', 'Embarked']
cat = train[categorical_features]
cat_t = test[categorical_features]
onehotencoder = OneHotEncoder()
X = onehotencoder.fit_transform(cat).toarray()
Xt =  onehotencoder.fit_transform(cat_t).toarray()

In [ ]:
X = np.concatenate([X, Age_discretized_train.toarray(), Fare_discretized_train.toarray()], axis=1)
Xt = np.concatenate((Xt, Age_discretized_test.toarray(), Fare_discretized_test.toarray()), axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score

model = LinearRegression()
model.fit(X_train, y_train)
yp = model.predict(X_train)

print(mean_squared_error(yp, y_train))
best_mse = 10000
best_i = 0
for i in range(2000):
    yp1 = yp > i/1000
    r = mean_squared_error(yp1, y_train)
    if r < best_mse:
        best_mse = r
        best_i = i
        # print(best_mse)
print(best_mse)
print(best_i)
yp1 = yp > best_i/1000
print(accuracy_score(y_train, yp1))
yp = model.predict(X_test)

print(mean_squared_error(yp, y_test))
yp1 = yp > best_i/1000
print(mean_squared_error(yp1, y_test))
print(accuracy_score(y_test, yp1))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import TheilSenRegressor, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from xgboost import XGBClassifier
acc_train = []
acc_test = []


In [ ]:
# model = XGBClassifier(learning_rate=0.05 + 4*0.01)
model = XGBClassifier(n_jobs=-1)
model.fit(X, y)

yp = model.predict(Xt)

with open('pred.csv', 'w') as out, open('/kaggle/input/tabular-playground-series-apr-2021/test.csv') as inp:
    out.write('PassengerId,Survived\n')
    inp.readline()
    for v in yp:
        line = inp.readline().split(",")[0]
        out.write(line + "," + str(int(v)) + "\n")

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import TheilSenRegressor, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from xgboost import XGBClassifier
acc_train = []
acc_test = []

# model = XGBClassifier(learning_rate=0.5)


model = XGBClassifier(base_score=0.1, gamma=1, min_child_weight=3, max_depth=5, n_jobs=-1)
model.fit(X_train, y_train)
yp = model.predict(X_train)
acc_train = accuracy_score(yp, y_train)

yp = model.predict(X_test)
acc_test = accuracy_score(yp, y_test)

print(acc_train)
print(acc_test)
